In [1]:
import googlemaps
from datetime import datetime

In [2]:
import json
import requests
from urllib.request import urlopen
import csv
import time
from urllib.parse import urlencode

In [20]:
#Grabbing and parsing the JSON data
def GoogPlac(city, business, pricemin,pricemax,nexter,keys):
  #making the url
    if (nexter != 'a'):
        MyUrl = ('https://maps.googleapis.com/maps/api/place/textsearch/json'
           '?pagetoken=%s'
           '&key=%s') % (nexter, keys)
    else:
        MyUrl = ('https://maps.googleapis.com/maps/api/place/textsearch/json'
                    '?query=%s'
                     '&type=%s'
                     '&minprice=%s'
                     '&maxprice=%s'
                     '&key=%s') % (city, business, pricemin, pricemax, keys)

  #grabbing the JSON result
    response = urlopen(MyUrl)
    jsonRaw = response.read()
    jsonData = json.loads(jsonRaw)
    return jsonData

In [21]:
place = 'New York'
business = 'restaurant'
key = 'Insert Key Here'

data_file = open('restaurants2.csv', 'w')
csvwriter = csv.writer(data_file)
full_data = GoogPlac(place,business,0,4,'a',key)
sub_result = full_data['results']
count = 0


csvwriter.writerow(['Name','Latitude','Longitude'])

while(full_data['status'] == 'OK' and count < 3):
    try:
        for i in range(0,len(sub_result)):
            csvwriter.writerow([sub_result[i]['name'], sub_result[i]['geometry']['location']['lat'], 
                       sub_result[i]['geometry']['location']['lng']])
        time.sleep(5)
        count += 1
        full_data = GoogPlac(place,business,0,4,full_data['next_page_token'],key)
        sub_result = full_data['results']
    except KeyError:
        break
    


if (full_data['status'] == 'OK' and count < 3):
        GoogPlac(place,business,0,4,'a',key)

    
data_file.close()

{'html_attributions': [],
 'next_page_token': 'CqQDnAEAAJiZFuox1bB5DWGvoTweT4RDnaMdGYsKwvO6S9HXuDPzjw8vckRROyxQ6ylcodsuKR_ePLEzCl1DqpWD17T6VGrV5t-CV2jCronPAGozbO7JJkZYrSiogAoJwWlIbYqVW9NDrltn6RgfSXXE_k5tWXx6QzbLjLt5PMlYs5_vBagsn_7irUBrYlumYAE59SoVKwTtPNOd2Ra6Qr65iQAV3Q0XtAR8iLh6A_YqiLwPhcjQlvRfGMejI0C48vAMJN8D-R4F6E3GwOpPnCH6BueY3pvrxf0vcc5YIy8V1qEYR5pmLJKBwIzezbIJEuzPa1wAUGY0kOx2dhXkql4MgIdwYVePb9ENLtttgdn05X9DseaAgseH1n7nC94URHpDhAwvKt4SgcZA3-iT3OnJQMsly--9vWGblAvGef9ZMm2Cze5fQEmEfZjn6TwjVXvbUeNhXuww6xV4XUbsAXKSspzbdE2IQJnYP6G4-TioWbbNfNXW83hk-i_HcWwuDhK8iQVH-aCU27a-c4KAx2DX_CSzGqKZKQFmzq680gVh95OTCi3SEhANsFTL3SkAyGyoVIu7k-sdGhQzVS60OuptOZEZD6LZSje58xnImA',
 'results': [{'formatted_address': 'Plot 1, Vivanta by Taj NCR, Sector 44, Gurugram, Haryana 122004, India',
   'geometry': {'location': {'lat': 28.455887, 'lng': 77.0705242},
    'viewport': {'northeast': {'lat': 28.4572359802915,
      'lng': 77.0718731802915},
     'southwest': {'lat': 28.45453801970849, 'lng': 77.0691752197084